In [103]:
import face_recognition as fr
import pandas as pd
from os import listdir, path, getcwd, mkdir

In [104]:
train_images_path = "./images/train"
test_images_path = "./images/test"
gathered_landmarks = ['chin', 'left_eyebrow', 'right_eyebrow', 'nose_bridge', 'nose_tip', 'left_eye', 'right_eye',
                     'top_lip', 'bottom_lip']
emotions = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']

In [105]:
image = fr.load_image_file("images/train/Anger/trainAnger001_031.jpg")
face_landmarks_list = fr.face_landmarks(image)

In [106]:
face_landmarks_list

[{'chin': [(103, 124),
   (105, 139),
   (108, 152),
   (111, 166),
   (116, 179),
   (126, 190),
   (139, 198),
   (155, 203),
   (172, 203),
   (190, 200),
   (204, 193),
   (216, 184),
   (225, 171),
   (228, 157),
   (228, 142),
   (228, 128),
   (227, 113)],
  'left_eyebrow': [(111, 113), (119, 105), (130, 102), (142, 102), (153, 104)],
  'right_eyebrow': [(167, 101), (178, 97), (190, 96), (202, 97), (212, 104)],
  'nose_bridge': [(161, 113), (162, 121), (163, 129), (163, 138)],
  'nose_tip': [(152, 149), (158, 150), (165, 150), (172, 148), (178, 146)],
  'left_eye': [(123, 120),
   (130, 115),
   (139, 114),
   (147, 119),
   (139, 121),
   (131, 122)],
  'right_eye': [(178, 116),
   (185, 110),
   (194, 109),
   (201, 113),
   (195, 116),
   (187, 117)],
  'top_lip': [(143, 170),
   (152, 163),
   (160, 159),
   (166, 160),
   (172, 157),
   (181, 160),
   (192, 165),
   (187, 165),
   (173, 164),
   (166, 165),
   (160, 165),
   (147, 170)],
  'bottom_lip': [(192, 165),
   (182

In [137]:
def append_to_df(data, columns, df):
    if 'test' not in data[0]:
        for emotion in emotions:
            columns.append('is' + emotion)
            if emotion in data[0]:
                data.append(1)
            else:
                data.append(0)
    df = df.append(pd.DataFrame([data], columns=columns), ignore_index=True)
    return df


def data_extraction(images_path):
    person = ""
    data = []
    columns = ['video']
    df = pd.DataFrame()
    for image in sorted(listdir(images_path)):
        if not '.jpg' in image:
            continue
            
        if person == image.split('_')[0]:
            warned = False
            new_frame = image.split('_')[1].split('.')[0]
            new_landmarks = fr.face_landmarks(fr.load_image_file(path.join(images_path, image)))
            if len(landmarks) == 0:
                landmarks = new_landmarks
            for gathered_landmark in gathered_landmarks:
                for idx, point in enumerate(landmarks[0][gathered_landmark]):
                    try:
                        if len(new_landmarks[0][gathered_landmark]) != len(landmarks[0][gathered_landmark]):
                            print(" WARN: can't find the proper number of points for image: " + image)
                            print(" WARN: landmark: " + gathered_landmark)
                            print(" WARN: number of points: ", len(new_landmarks[0][gathered_landmark]))
                            print(" WARN: number of expected points: ", len(landmarks[0][gathered_landmark]))
                   
                        distanceX = point[0] - new_landmarks[0][gathered_landmark][idx][0]
                        distanceY = point[1] - new_landmarks[0][gathered_landmark][idx][1]
                    
                        columns.append(str(frame_count) + '-' + str(frame_count+1) + '_' + gathered_landmark + str(idx) + 'X')
                        data.append(distanceX)

                        columns.append(str(frame_count) + '-' + str(frame_count+1) + '_' + gathered_landmark + str(idx) + 'Y')
                        data.append(distanceY)
                    except:
                        if not warned:
                            print(" WARN: can't find the proper landmarks for image: " + image)
                            print(" WARN: found landmarks: ", new_landmarks)
                            print(" WARN: searching for: " + gathered_landmark)
                            warned = True
                        columns.append(str(frame_count) + '-' + str(frame_count+1) + '_' + gathered_landmark + str(idx) + 'X')
                        data.append(0)
                        columns.append(str(frame_count) + '-' + str(frame_count+1) + '_' + gathered_landmark + str(idx) + 'Y')
                        data.append(0)
                        
            landmarks = new_landmarks
            frame = new_frame
            frame_count+=1
        else:
            if len(data) != 0:
                df = append_to_df(data, columns, df)
            landmarks = fr.face_landmarks(fr.load_image_file(path.join(images_path, image)))
            person = image.split('_')[0]
            frame = image.split('_')[1].split('.')[0]
            frame_count = 0
            print(person, frame)
            data = [person]
            columns = ['video']
            
    df = append_to_df(data, columns, df)
    return df

In [138]:
if __name__ == '__main__':
    df = pd.DataFrame()
    for emotion in sorted(listdir(train_images_path)):
        if not path.isdir(path.join(train_images_path, emotion)):
            continue
        print("===================")
        print("Processing emotion: " + emotion)

        df = df.append(data_extraction(path.join(train_images_path, emotion)), ignore_index=True)
        
    df.to_csv('train.csv')
    print("|=|+|+|+|+|+|+|+|++||++|+|+|")
    print("Processing test videos: ")
    df = pd.DataFrame()
    df = data_extraction(path.join(test_images_path))
    df.to_csv('test.csv')

|=|+|+|+|+|+|+|+|++||++|+|+|
Processing test videos: 
test001 030
Processed count of frames:  2  frame number:  060
Processed count of frames:  3  frame number:  090
Processed count of frames:  4  frame number:  120
test002 040
Processed count of frames:  2  frame number:  080
Processed count of frames:  3  frame number:  120
Processed count of frames:  4  frame number:  160
test003 025
Processed count of frames:  2  frame number:  050
Processed count of frames:  3  frame number:  075
Processed count of frames:  4  frame number:  100
test004 029
Processed count of frames:  2  frame number:  058
Processed count of frames:  3  frame number:  087
Processed count of frames:  4  frame number:  116
test005 031
Processed count of frames:  2  frame number:  062
Processed count of frames:  3  frame number:  093
Processed count of frames:  4  frame number:  124
test006 036
Processed count of frames:  2  frame number:  072
Processed count of frames:  3  frame number:  108
Processed count of frame

Processed count of frames:  4  frame number:  136
test051 044
Processed count of frames:  2  frame number:  088
Processed count of frames:  3  frame number:  132
Processed count of frames:  4  frame number:  176
test052 033
Processed count of frames:  2  frame number:  066
Processed count of frames:  3  frame number:  099
Processed count of frames:  4  frame number:  132
test053 036
Processed count of frames:  2  frame number:  072
Processed count of frames:  3  frame number:  108
Processed count of frames:  4  frame number:  144
test054 033
Processed count of frames:  2  frame number:  066
Processed count of frames:  3  frame number:  099
Processed count of frames:  4  frame number:  132
test055 036
Processed count of frames:  2  frame number:  072
Processed count of frames:  3  frame number:  108
Processed count of frames:  4  frame number:  144
test056 035
Processed count of frames:  2  frame number:  070
Processed count of frames:  3  frame number:  105
Processed count of frames:  

Processed count of frames:  4  frame number:  116
test102 031
Processed count of frames:  2  frame number:  062
Processed count of frames:  3  frame number:  093
Processed count of frames:  4  frame number:  124
test103 030
Processed count of frames:  2  frame number:  060
Processed count of frames:  3  frame number:  090
Processed count of frames:  4  frame number:  120
test104 032
Processed count of frames:  2  frame number:  064
Processed count of frames:  3  frame number:  096
Processed count of frames:  4  frame number:  128
test105 032
Processed count of frames:  2  frame number:  064
Processed count of frames:  3  frame number:  096
Processed count of frames:  4  frame number:  128
test106 032
Processed count of frames:  2  frame number:  064
Processed count of frames:  3  frame number:  096
Processed count of frames:  4  frame number:  128
test107 028
Processed count of frames:  2  frame number:  056
Processed count of frames:  3  frame number:  084
Processed count of frames:  

Processed count of frames:  4  frame number:  092
test153 024
Processed count of frames:  2  frame number:  048
Processed count of frames:  3  frame number:  072
Processed count of frames:  4  frame number:  096
test154 028
Processed count of frames:  2  frame number:  056
Processed count of frames:  3  frame number:  084
Processed count of frames:  4  frame number:  112
test155 024
Processed count of frames:  2  frame number:  048
Processed count of frames:  3  frame number:  072
Processed count of frames:  4  frame number:  096
test156 021
Processed count of frames:  2  frame number:  042
Processed count of frames:  3  frame number:  063
Processed count of frames:  4  frame number:  084
test157 023
Processed count of frames:  2  frame number:  046
Processed count of frames:  3  frame number:  069
Processed count of frames:  4  frame number:  092
test158 021
Processed count of frames:  2  frame number:  042
Processed count of frames:  3  frame number:  063
Processed count of frames:  

Processed count of frames:  4  frame number:  124
test204 025
Processed count of frames:  2  frame number:  050
Processed count of frames:  3  frame number:  075
Processed count of frames:  4  frame number:  100
test205 028
Processed count of frames:  2  frame number:  056
Processed count of frames:  3  frame number:  084
Processed count of frames:  4  frame number:  112
test206 025
Processed count of frames:  2  frame number:  050
Processed count of frames:  3  frame number:  075
Processed count of frames:  4  frame number:  100
test207 025
Processed count of frames:  2  frame number:  050
Processed count of frames:  3  frame number:  075
Processed count of frames:  4  frame number:  100
test208 028
Processed count of frames:  2  frame number:  056
Processed count of frames:  3  frame number:  084
Processed count of frames:  4  frame number:  112
test209 028
Processed count of frames:  2  frame number:  056
Processed count of frames:  3  frame number:  084
Processed count of frames:  

Processed count of frames:  4  frame number:  128
test255 032
Processed count of frames:  2  frame number:  064
Processed count of frames:  3  frame number:  096
Processed count of frames:  4  frame number:  128
test256 032
Processed count of frames:  2  frame number:  064
Processed count of frames:  3  frame number:  096
Processed count of frames:  4  frame number:  128
test257 032
Processed count of frames:  2  frame number:  064
Processed count of frames:  3  frame number:  096
Processed count of frames:  4  frame number:  128
test258 033
Processed count of frames:  2  frame number:  066
Processed count of frames:  3  frame number:  099
Processed count of frames:  4  frame number:  132
test259 031
Processed count of frames:  2  frame number:  062
Processed count of frames:  3  frame number:  093
Processed count of frames:  4  frame number:  124
test260 032
Processed count of frames:  2  frame number:  064
Processed count of frames:  3  frame number:  096
Processed count of frames:  

In [139]:
df. head()

,video,0-1_chin0X,0-1_chin0Y,0-1_chin1X,0-1_chin1Y,0-1_chin2X,0-1_chin2Y,0-1_chin3X,0-1_chin3Y,0-1_chin4X,...,2-3_bottom_lip7X,2-3_bottom_lip7Y,2-3_bottom_lip8X,2-3_bottom_lip8Y,2-3_bottom_lip9X,2-3_bottom_lip9Y,2-3_bottom_lip10X,2-3_bottom_lip10Y,2-3_bottom_lip11X,2-3_bottom_lip11Y
0,test001,-1,-1,-1,-2,-1,-3,-1,-3,-1,...,-2,-1,-3,-1,-4,-1,-4,-1,-6,-2
1,test002,0,-2,0,-1,1,0,0,1,-1,...,2,-7,0,-7,0,-6,0,-6,-2,-5
2,test003,1,-2,1,0,1,0,0,0,1,...,-5,-17,-5,-17,-5,-17,-4,-17,-6,-16
3,test004,-1,1,-1,1,-1,1,-1,1,-1,...,7,11,7,14,9,13,9,13,9,11
4,test005,0,-2,0,-1,1,-2,0,-1,1,...,-1,-16,-1,-14,-2,-14,-2,-15,-3,-19
